# Mitiq Tutorial

Before running this tutorial, make sure you have `cirq`, and `mitiq` installed.

```bash
pip install cirq mitiq
```

Use `pip list | grep $package` to ensure it is installed.


# Goals

- learn how to apply ZNE on a basic workflow
- explore other QEM methods via calibration

In [ ]:
%pip install mitiq

## Executors

Executors are python functions that consume a quantum circuit, and output an expectation value.
A type signature might look something like `Circuit -> float`.
That said, executors can have additional arguments used to control other parts of the execution process.
The circuit must be the first argument, however.

In [ ]:
import cirq


def execute(circuit, noise_level=0.005):
    """Returns Tr[ρ |0⟩⟨0|] where ρ is the state prepared by the circuit
    with depolarizing noise."""
    # add depolarizing noise
    return (
        cirq.DensityMatrixSimulator()
        .simulate(circuit)
        .final_density_matrix[0, 0]
        .real
    )

## ZNE

First, we define a simple circuit to work with.

In [ ]:
a, b, c = cirq.LineQubit.range(3)

circuit = cirq.Circuit([
    cirq.H(a),
    cirq.CNOT(a, b),
    cirq.CNOT(b, c),
    cirq.S(a),
])

In [ ]:
print(circuit)

### Unitary Folding

In [ ]:
from mitiq.zne.scaling import fold_gates_at_random, fold_global


folded_circuit = "?"

print(folded_circuit)

Randomized Benchmarking circuits are circuits that are in effect, equivalent to the identity. Hence, the ideal probability that the end state is $|00\cdots 0\rangle$ is 1.

In [ ]:
from mitiq.benchmarks import generate_rb_circuits

In [ ]:
from mitiq import zne


# circuit = generate_rb_circuits(2, num_cliffords=20)[0]

true_value = execute(circuit, noise_level=0.0)
noisy_value = execute(circuit)
zne_value = zne.execute_with_zne(circuit, execute)

print(f"Error w/o  Mitiq: {abs((true_value - noisy_value) / true_value):.3f}")
print(f"Error w Mitiq:    {abs((true_value - zne_value) / true_value):.3f}")

## Calibration

When you want to explore more options

In [ ]:
import numpy as np
from mitiq import MeasurementResult


def execute_with_depolarizing(circuit, noise_level=0.001):
    """Return bitstrings outcomes from circuit executions performed by
    a density matrix simulation with depolarizing noise."""
    circuit = circuit.with_noise(cirq.depolarize(p=noise_level))
    result = cirq.DensityMatrixSimulator().run(circuit, repetitions=100)
    bitstrings = np.column_stack(list(result.measurements.values()))
    return MeasurementResult(bitstrings)

In [ ]:
from mitiq import Calibrator

cal = Calibrator(...)

cal.run(log=True)

In [ ]:
cal.best_strategy()

In [ ]:
from mitiq.calibration import PECSettings

# run calibrator with PEC
cal = Calibrator(execute_with_depolarizing, frontend="cirq", settings=PECSettings)

In [ ]:
cal.run(log=True)